# Setup Environment

In [ ]:
%load_ext autoreload
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path_to_add = os.path.join(parent_dir, "src")
sys.path.insert(0, path_to_add)

# Get Data

## Free text comments

In [ ]:
import requests
import pandas as pd
import janitor

# URL of the endpoint
url = "https://data.austintexas.gov/resource/jeyv-db9u.json"

# Fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(response.json()).clean_names()
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

## Questions data (likert)

In [ ]:
url_likert = "https://data.austintexas.gov/resource/s2py-ceb7.json"
data_dictionary = "https://data.austintexas.gov/api/views/s2py-ceb7/columns.json"

response = requests.get(url_likert)
if response.status_code == 200:
    # Convert the JSON data to a pandas DataFrame
    dfq = pd.DataFrame(response.json())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

In [ ]:
likert_columns = ['the_city_of_austin_as_a_place_1', 'the_city_of_austin_as_a_place_2',
       'the_city_of_austin_as_a_place_3', 'overall_quality_of_life_in',]

column_rename = {'the_city_of_austin_as_a_place_1' : 'place to work',
                 'the_city_of_austin_as_a_place_2' : 'place to raise children',
                 'the_city_of_austin_as_a_place_3' : 'place to retire',
                 'overall_quality_of_life_in' : 'overall quality of life in the city'}

dfq = dfq.rename_columns(column_rename)

likert_encoded = ["likert_encoded_" + col for col in column_rename.values()]

In [ ]:
dfq.columns

In [ ]:
%autoreload 2
from pandas_survey_toolkit import analytics, nlp, vis
from pandas_survey_toolkit.vis import cluster_heatmap_plot


In [ ]:
dfq2 = dfq.cluster_questions(column_rename.values())

In [ ]:
cluster_heatmap_plot(dfq2, x="question_cluster_id", y = likert_encoded)

In [ ]:
dfq.cluster_questions()

In [ ]:
dfq.to_excel('austin_likert.xlsx')

In [3]:
%autoreload 2
from pandas_survey_toolkit import analytics, nlp, vis
from pandas_survey_toolkit.vis import create_keyword_graph, visualize_keyword_graph,visualize_keyword_graph_force, create_keyword_sentiment_df_simple, create_sentiment_color_mapping

c:\Users\LONAL23\OneDrive - PA Consulting Group\7 - Data Scientist\pandas-survey-toolkit\.venv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [13]:
df2 = df2.extract_keywords("response", top_n=4, ngram_range=(1,1), min_df=4, keyword_min_count=5)#.extract_sentiment("response")

In [14]:
from functools import reduce
kw = reduce(lambda x,y : x | y, df2.refined_keywords.map(lambda x: set(x)))
kw_old = reduce(lambda x,y : x | y, df2.keywords.map(lambda x: set(x)))

In [16]:
len(kw)

103

In [18]:
sentiment_df = create_keyword_sentiment_df_simple(df2)
sentiment_df = sentiment_df.query("word != 'not'")
color_mapping = create_sentiment_color_mapping(sentiment_df)

G = create_keyword_graph(df2, 'keywords', node_color_mapping=color_mapping)
G.remove_node("not")
visualize_keyword_graph_force(G, output_file='keyword_sentiment_graph.html', colormap='RdBu', min_edge_count=2, min_node_count=1)

Graph saved to keyword_sentiment_graph.html
